# System Price Forecasting - Monthly Rolling Window

- System price forecasting in the British BM across several short-term time horizons
- Comparison of forecasting performance with REMIT generator unavailability data vs without 
- 2023 data used for model training
- 2024 data used for model testing
- Rolling monthly window forecasting, with performance averaged across 2024 test data
- Previous months data added to next model training set, e.g. After testing on January 2024, January 2024 becomes part of test set for February 2024 and following forecasts 
- XGBoost regression model with optuna hyperparameter tuning
- Model evaluation using MAE, RMSE, R2
- Forecast function runs the model, takes one input which is the df

## Market Data Load

In [ ]:
# Load libraries
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import math
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna
import joblib
import os

In [ ]:
# Load baseline market dataset
market_data = pd.read_csv("Final_Dataset_Forward_Forecasts_2021_2024.csv")

# Convert 'StartTime' to datetime
market_data['StartTime'] = pd.to_datetime(market_data['StartTime'])

# Display head of the dataset
market_data.head()

In [ ]:
# Trim data for 2023 onwards
market_data = market_data[market_data['StartTime'] >= '2023-01-01']

# Reset index
market_data.reset_index(drop=True, inplace=True)

# Drop SettlementPeriod column
market_data.drop(columns=['SettlementPeriod'], inplace=True)

In [ ]:
# Plot SystemSellPrice 
plt.figure(figsize=(12, 6))
plt.plot(market_data['StartTime'], market_data['SystemPrice'], label='System Sell Price', color='blue')
plt.title('UK Balancing Market System Price (2023-2024)')
plt.xlabel('Date')
plt.ylabel('Price (£/MWh)')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Add time of day, day of week, month, and season as features
market_data['TimeOfDay'] = market_data['StartTime'].dt.hour
market_data['DayOfWeek'] = market_data['StartTime'].dt.dayofweek
market_data['Month'] = market_data['StartTime'].dt.month
market_data['Season'] = market_data['StartTime'].dt.month % 12 // 3 + 1

In [ ]:
# Exclude columns
columns_to_plot = [col for col in market_data.columns 
                   if col not in ['StartTime', 'SettlementPeriod', 'SystemPrice']]

# Calculate grid size
n = len(columns_to_plot)
n_cols = 3  # Number of columns in the subplot grid
n_rows = math.ceil(n / n_cols)

# Create subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 4 * n_rows), sharex=True)
axes = axes.flatten()  # Flatten in case it's a 2D array

# Plot each column
for i, col in enumerate(columns_to_plot):
    axes[i].plot(market_data['StartTime'], market_data[col])
    axes[i].set_title(col)
    axes[i].grid(True)
    axes[i].tick_params(axis='x', rotation=45)

# Turn off any unused axes
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Print SysmenPrice statistics
print("System Price Statistics:")
print(market_data['SystemPrice'].describe())

## Individual Generator Data Load

In [ ]:
# Load datasets
individual_unit_capacity = pd.read_csv("Individual_Unit_Capacity.csv")
individual_unit_capacity_planned = pd.read_csv("Individual_Unit_Capacity_Planned.csv")
individual_unit_capacity_unplanned = pd.read_csv("Individual_Unit_Capacity_Unplanned.csv")

In [ ]:
# Trim data for 2023 onwards
individual_unit_capacity['StartTime'] = pd.to_datetime(individual_unit_capacity['StartTime'])
individual_unit_capacity = individual_unit_capacity[individual_unit_capacity['StartTime'] >= '2023-01-01']

individual_unit_capacity_planned['StartTime'] = pd.to_datetime(individual_unit_capacity_planned['StartTime'])
individual_unit_capacity_planned = individual_unit_capacity_planned[individual_unit_capacity_planned['StartTime'] >= '2023-01-01']

individual_unit_capacity_unplanned['StartTime'] = pd.to_datetime(individual_unit_capacity_unplanned['StartTime'])
individual_unit_capacity_unplanned = individual_unit_capacity_unplanned[individual_unit_capacity_unplanned['StartTime'] >= '2023-01-01']

# Reset index
individual_unit_capacity.reset_index(drop=True, inplace=True)
individual_unit_capacity_planned.reset_index(drop=True, inplace=True)
individual_unit_capacity_unplanned.reset_index(drop=True, inplace=True)

In [ ]:
# Merge market data with generator data
generator_data_merged = individual_unit_capacity.merge(market_data, on='StartTime', how='inner')
generator_data_merged_planned = individual_unit_capacity_planned.merge(market_data, on='StartTime', how='inner')
generator_data_merged_unplanned = individual_unit_capacity_unplanned.merge(market_data, on='StartTime', how='inner') 

In [ ]:
# Load Top 50 SHAP features CSV files
top_50_fuel = pd.read_csv("Top_50_Fuel_Types.csv")
top_50_fuel_planned = pd.read_csv("Top_50_Fuel_Types_Planned.csv")
top_50_fuel_unplanned = pd.read_csv("Top_50_Fuel_Types_Unplanned.csv")

## Time Horizon Data

In [ ]:
# 1 hour ahead forecast 
baseline_data_1hr = market_data.copy()

# Drop lagged DeratedMargin and LossLossOfLoadProbability columns
baseline_data_1hr.drop(columns=['LossOfLoadProbability_2','LossOfLoadProbability_4','LossOfLoadProbability_8','LossOfLoadProbability_12',
                          'DeratedMargin_2','DeratedMargin_4','DeratedMargin_8','DeratedMargin_12'], inplace=True)

# Create 1 hour lags for SystemPrice, NetImbalanceVolume and InterconnectorFlow
baseline_data_1hr['SystemPrice_1hr'] = baseline_data_1hr['SystemPrice'].shift(2)
baseline_data_1hr['NetImbalanceVolume_1hr'] = baseline_data_1hr['NetImbalanceVolume'].shift(2)
baseline_data_1hr['InterconnectorFlow_1hr'] = baseline_data_1hr['InterconnectorFlow'].shift(2)

# Rename DeratedMargin_1 and LossOfLoadProbability_1 columns
baseline_data_1hr.rename(columns={'DeratedMargin_1': 'DeratedMargin_1hr', 'LossOfLoadProbability_1': 'LossOfLoadProbability_1hr'}, inplace=True)

# Drop non lagged columns
baseline_data_1hr.drop(columns=['NetImbalanceVolume', 'InterconnectorFlow'], inplace=True)

# Drop rows with missing values
baseline_data_1hr = baseline_data_1hr.dropna()

# Merge baseline data with Generator dataset
generator_data_1hr = individual_unit_capacity.merge(baseline_data_1hr, on='StartTime', how='left')
generator_data_planned_1hr = individual_unit_capacity_planned.merge(baseline_data_1hr, on='StartTime', how='left')
generator_data_unplanned_1hr = individual_unit_capacity_unplanned.merge(baseline_data_1hr, on='StartTime', how='left')

# Drop rows with missing values
generator_data_1hr = generator_data_1hr.dropna()
generator_data_planned_1hr = generator_data_planned_1hr.dropna()
generator_data_unplanned_1hr = generator_data_unplanned_1hr.dropna()

In [ ]:
# 2 hour ahead forecast 
baseline_data_2hr = market_data.copy()

# Drop lagged DeratedMargin and LossLossOfLoadProbability columns
baseline_data_2hr.drop(columns=['LossOfLoadProbability_1','LossOfLoadProbability_4','LossOfLoadProbability_8','LossOfLoadProbability_12',
                          'DeratedMargin_1','DeratedMargin_4','DeratedMargin_8','DeratedMargin_12'], inplace=True)

# Create 1 hour lags for SystemPrice, NetImbalanceVolume and InterconnectorFlow
baseline_data_2hr['SystemPrice_2hr'] = baseline_data_2hr['SystemPrice'].shift(4)
baseline_data_2hr['NetImbalanceVolume_2hr'] = baseline_data_2hr['NetImbalanceVolume'].shift(4)
baseline_data_2hr['InterconnectorFlow_2hr'] = baseline_data_2hr['InterconnectorFlow'].shift(4)

# Rename DeratedMargin_2 and LossOfLoadProbability_2 columns
baseline_data_2hr.rename(columns={'DeratedMargin_2': 'DeratedMargin_2hr', 'LossOfLoadProbability_2': 'LossOfLoadProbability_2hr'}, inplace=True)

# Drop non lagged columns
baseline_data_2hr.drop(columns=['NetImbalanceVolume', 'InterconnectorFlow'], inplace=True)

# Drop rows with missing values
baseline_data_2hr = baseline_data_2hr.dropna()

# Merge baseline data with Generator dataset
generator_data_2hr = individual_unit_capacity.merge(baseline_data_2hr, on='StartTime', how='left')
generator_data_planned_2hr = individual_unit_capacity_planned.merge(baseline_data_2hr, on='StartTime', how='left')
generator_data_unplanned_2hr = individual_unit_capacity_unplanned.merge(baseline_data_2hr, on='StartTime', how='left')

# Drop rows with missing values
generator_data_2hr = generator_data_2hr.dropna()
generator_data_planned_2hr = generator_data_planned_2hr.dropna()
generator_data_unplanned_2hr = generator_data_unplanned_2hr.dropna()

In [ ]:
# 4 hour ahead forecast 
baseline_data_4hr = market_data.copy()

# Drop lagged DeratedMargin and LossLossOfLoadProbability columns
baseline_data_4hr.drop(columns=['LossOfLoadProbability_1','LossOfLoadProbability_2','LossOfLoadProbability_8','LossOfLoadProbability_12',
                          'DeratedMargin_1','DeratedMargin_2','DeratedMargin_8','DeratedMargin_12'], inplace=True)

# Create 1 hour lags for SystemPrice, NetImbalanceVolume and InterconnectorFlow
baseline_data_4hr['SystemPrice_4hr'] = baseline_data_4hr['SystemPrice'].shift(8)
baseline_data_4hr['NetImbalanceVolume_4hr'] = baseline_data_4hr['NetImbalanceVolume'].shift(8)
baseline_data_4hr['InterconnectorFlow_4hr'] = baseline_data_4hr['InterconnectorFlow'].shift(8)

# Rename DeratedMargin_2 and LossOfLoadProbability_2 columns
baseline_data_4hr.rename(columns={'DeratedMargin_4': 'DeratedMargin_4hr', 'LossOfLoadProbability_4': 'LossOfLoadProbability_4hr'}, inplace=True)

# Drop non lagged columns
baseline_data_4hr.drop(columns=['NetImbalanceVolume', 'InterconnectorFlow'], inplace=True)

# Drop rows with missing values
baseline_data_4hr = baseline_data_4hr.dropna()

# Merge baseline data with Generator dataset
generator_data_4hr = individual_unit_capacity.merge(baseline_data_4hr, on='StartTime', how='left')
generator_data_planned_4hr = individual_unit_capacity_planned.merge(baseline_data_4hr, on='StartTime', how='left')
generator_data_unplanned_4hr = individual_unit_capacity_unplanned.merge(baseline_data_4hr, on='StartTime', how='left')

# Drop rows with missing values
generator_data_4hr = generator_data_4hr.dropna()
generator_data_planned_4hr = generator_data_planned_4hr.dropna()
generator_data_unplanned_4hr = generator_data_unplanned_4hr.dropna()

In [ ]:
# 8 hour ahead forecast 
baseline_data_8hr = market_data.copy()

# Drop lagged DeratedMargin and LossLossOfLoadProbability columns
baseline_data_8hr.drop(columns=['LossOfLoadProbability_1','LossOfLoadProbability_2','LossOfLoadProbability_4','LossOfLoadProbability_12',
                          'DeratedMargin_1','DeratedMargin_2','DeratedMargin_4','DeratedMargin_12'], inplace=True)

# Create 1 hour lags for SystemPrice, NetImbalanceVolume and InterconnectorFlow
baseline_data_8hr['SystemPrice_8hr'] = baseline_data_8hr['SystemPrice'].shift(16)
baseline_data_8hr['NetImbalanceVolume_8hr'] = baseline_data_8hr['NetImbalanceVolume'].shift(16)
baseline_data_8hr['InterconnectorFlow_8hr'] = baseline_data_8hr['InterconnectorFlow'].shift(16)

# Rename DeratedMargin_2 and LossOfLoadProbability_2 columns
baseline_data_8hr.rename(columns={'DeratedMargin_8': 'DeratedMargin_8hr', 'LossOfLoadProbability_8': 'LossOfLoadProbability_8hr'}, inplace=True)

# Drop non lagged columns
baseline_data_8hr.drop(columns=['NetImbalanceVolume', 'InterconnectorFlow'], inplace=True)

# Drop rows with missing values
baseline_data_8hr = baseline_data_8hr.dropna()

# Merge baseline data with Generator dataset
generator_data_8hr = individual_unit_capacity.merge(baseline_data_8hr, on='StartTime', how='left')
generator_data_planned_8hr = individual_unit_capacity_planned.merge(baseline_data_8hr, on='StartTime', how='left')
generator_data_unplanned_8hr = individual_unit_capacity_unplanned.merge(baseline_data_8hr, on='StartTime', how='left')

# Drop rows with missing values
generator_data_8hr = generator_data_8hr.dropna()
generator_data_planned_8hr = generator_data_planned_8hr.dropna()
generator_data_unplanned_8hr = generator_data_unplanned_8hr.dropna()

In [ ]:
# 12 hour ahead forecast 
baseline_data_12hr = market_data.copy()

# Drop lagged DeratedMargin and LossLossOfLoadProbability columns
baseline_data_12hr.drop(columns=['LossOfLoadProbability_1','LossOfLoadProbability_2','LossOfLoadProbability_4','LossOfLoadProbability_8',
                          'DeratedMargin_1','DeratedMargin_2','DeratedMargin_4','DeratedMargin_8'], inplace=True)

# Create 1 hour lags for SystemPrice, NetImbalanceVolume and InterconnectorFlow
baseline_data_12hr['SystemPrice_12hr'] = baseline_data_12hr['SystemPrice'].shift(24)
baseline_data_12hr['NetImbalanceVolume_12hr'] = baseline_data_12hr['NetImbalanceVolume'].shift(24)
baseline_data_12hr['InterconnectorFlow_12hr'] = baseline_data_12hr['InterconnectorFlow'].shift(24)

# Rename DeratedMargin_2 and LossOfLoadProbability_2 columns
baseline_data_12hr.rename(columns={'DeratedMargin_12': 'DeratedMargin_12hr', 'LossOfLoadProbability_12': 'LossOfLoadProbability_12hr'}, inplace=True)

# Drop non lagged columns
baseline_data_12hr.drop(columns=['NetImbalanceVolume', 'InterconnectorFlow'], inplace=True)

# Drop rows with missing values
baseline_data_12hr = baseline_data_12hr.dropna()

# Merge baseline data with Generator dataset
generator_data_12hr = individual_unit_capacity.merge(baseline_data_12hr, on='StartTime', how='left')
generator_data_planned_12hr = individual_unit_capacity_planned.merge(baseline_data_12hr, on='StartTime', how='left')
generator_data_unplanned_12hr = individual_unit_capacity_unplanned.merge(baseline_data_12hr, on='StartTime', how='left')

# Drop rows with missing values
generator_data_12hr = generator_data_12hr.dropna()
generator_data_planned_12hr = generator_data_planned_12hr.dropna()
generator_data_unplanned_12hr = generator_data_unplanned_12hr.dropna()

In [ ]:
# Sets of valid generators 
valid_generators_main      = set(individual_unit_capacity.columns) & set(top_50_fuel['Feature'])
valid_generators_planned   = set(individual_unit_capacity.columns) & set(top_50_fuel_planned['Feature'])
valid_generators_unplanned = set(individual_unit_capacity.columns) & set(top_50_fuel_unplanned['Feature'])


# Generator various datasets: All generator data, planned generator data, unplanned generator data, planned generator top 50, unplanned generator top 50
prefixes     = ['generator_data', 'generator_data_planned', 'generator_data_unplanned']
time_windows = ['1hr', '2hr', '4hr', '8hr', '12hr']

for prefix in prefixes:
    if prefix == 'generator_data':
        valid_generators = valid_generators_main
    elif prefix == 'generator_data_planned':
        valid_generators = valid_generators_planned
    else:  # 'generator_data_unplanned'
        valid_generators = valid_generators_unplanned

    for window in time_windows:
        name = f"{prefix}_{window}"
        df = globals()[name]

        # preserve market columns (e.g., StartTime, price, etc.)
        non_generator_cols = [c for c in df.columns if c not in individual_unit_capacity.columns or c == 'StartTime']

        # generator columns to keep for this dataset
        generator_cols = sorted(list(valid_generators & set(df.columns)))

        # rename rule only for planned/unplanned generator columns
        if prefix == 'generator_data_planned':
            rename_map = {c: f"{c}_planned" for c in generator_cols}
        elif prefix == 'generator_data_unplanned':
            rename_map = {c: f"{c}_unplanned" for c in generator_cols}
        else:
            rename_map = {}

        filtered_cols = generator_cols + non_generator_cols
        filtered_df = df[filtered_cols].rename(columns=rename_map)

        globals()[f"{name}_filtered"] = filtered_df

# --- Build combined (planned + unplanned) per horizon ---
for window in time_windows:
    # base market variables from the non-tagged dataset
    base = globals()[f"generator_data_{window}"]
    market_cols = [c for c in base.columns if c not in individual_unit_capacity.columns or c == 'StartTime']
    market_df = base[market_cols].copy()

    # get the suffixed, filtered generator sets
    planned_df   = globals()[f"generator_data_planned_{window}_filtered"]
    unplanned_df = globals()[f"generator_data_unplanned_{window}_filtered"]

    # take only the generator columns from these (avoid duplicating market cols)
    planned_gen_cols   = [c for c in planned_df.columns   if c not in market_cols]
    unplanned_gen_cols = [c for c in unplanned_df.columns if c not in market_cols]

    combined_df = pd.concat(
        [market_df, planned_df[planned_gen_cols], unplanned_df[unplanned_gen_cols]],
        axis=1
    )

    globals()[f"generator_data_planned_unplanned_{window}_filtered"] = combined_df

## XGBoost Forecasting Function

In [ ]:
def run_monthly_rolling_forecast(
    df: pd.DataFrame,
    target_col: str = 'SystemPrice',
    dt_col: str = 'StartTime',
    time_format: str = '%Y-%m-%d %H:%M:%S',
    n_trials: int = 100,
    random_state: int = 42,
    save_model_dir: str = 'models'
):
    os.makedirs(save_model_dir, exist_ok=True)
    
    df = df.copy()
    df[dt_col] = pd.to_datetime(df[dt_col], format=time_format)
    df['Month'] = df[dt_col].dt.to_period('M')
    
    test_months = pd.period_range('2024-01', '2024-12', freq='M')
    feature_names = df.drop(columns=[dt_col, target_col, 'Month']).columns.tolist()
    
    results = []
    feature_importances = []

    # --- Store full-year actuals and predictions ---
    full_year_data = []

    for i, month in enumerate(test_months):
        print(f"\n🔹 Month: {month}")
        train_data = df[df[dt_col] < month.start_time]
        test_data = df[df['Month'] == month]

        X_train = train_data.drop(columns=[dt_col, target_col, 'Month'])
        y_train = train_data[target_col].values

        X_test = test_data.drop(columns=[dt_col, target_col, 'Month'])
        y_test = test_data[target_col].values
        time_test = test_data[dt_col]

        # Optuna tuning
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
                'max_depth': trial.suggest_int('max_depth', 3, 12),
                'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1.0, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 10.0, log=True),
                'random_state': random_state,
                'n_jobs': -1
            }
            model = XGBRegressor(**params)
            model.fit(X_train, y_train)
            y_val_pred = model.predict(X_test)
            return np.sqrt(mean_squared_error(y_test, y_val_pred))  # return positive RMSE

        # ✅ minimize RMSE
        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=n_trials)
        best_params = study.best_params

        model = XGBRegressor(**best_params, random_state=random_state, n_jobs=-1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Save model
        joblib.dump(model, f"{save_model_dir}/model_{month}.joblib")

        # Metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        results.append({'Month': str(month), 'RMSE': rmse, 'MAE': mae, 'R2': r2})

        # Feature importance
        imp = model.feature_importances_
        feature_importances.append(imp)

        # Store full-year predictions
        month_df = pd.DataFrame({'Time': time_test, 'Actual': y_test, 'Forecast': y_pred})
        full_year_data.append(month_df)

        # Individual plot
        plt.figure(figsize=(10, 4))
        plt.plot(month_df['Time'], month_df['Actual'], label='Actual', linestyle='--', marker='o', alpha=0.6)
        plt.plot(month_df['Time'], month_df['Forecast'], label='Forecast', linestyle='-', marker='x', alpha=0.6)
        plt.title(f"Actual vs Forecast - {month}")
        plt.xlabel("Time")
        plt.ylabel(target_col)
        plt.legend()
        plt.tight_layout()
        plt.show()

    # --- Final full-year plot ---
    full_2024_df = pd.concat(full_year_data).sort_values('Time')
    plt.figure(figsize=(14, 6))
    plt.plot(full_2024_df['Time'], full_2024_df['Actual'], label='Actual', linestyle='--', alpha=0.7)
    plt.plot(full_2024_df['Time'], full_2024_df['Forecast'], label='Forecast', alpha=0.7)
    plt.title("SystemPrice Forecast vs Actual - Full Year 2024")
    plt.xlabel("Time")
    plt.ylabel(target_col)
    plt.legend()
    plt.tight_layout()
    plt.grid(True)
    plt.show()

    # Results table
    results_df = pd.DataFrame(results)
    avg_row = {
        'Month': 'Average',
        'RMSE': results_df['RMSE'].mean(),
        'MAE': results_df['MAE'].mean(),
        'R2': results_df['R2'].mean()
    }
    results_df = pd.concat([results_df, pd.DataFrame([avg_row])], ignore_index=True)
    print("\nPerformance Summary:\n", results_df)

    # Feature importance
    feature_df = pd.DataFrame(feature_importances, columns=feature_names)
    avg_feature_importance = feature_df.mean().sort_values(ascending=False)

    # Plot average feature importances
    plt.figure(figsize=(10, 6))
    avg_feature_importance.plot(kind='barh')
    plt.title("Average Feature Importances (2024)")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

    return results_df, feature_df, full_2024_df

## Exact Forecast 

In [ ]:
results_baseline, fi_baseline, predictions_baseline = run_monthly_rolling_forecast(market_data)

In [ ]:
results_generator_data, fi_generator_data, predictions_generator_data = run_monthly_rolling_forecast(generator_data_merged)

## 1hr Ahead Forecast

In [ ]:
results_1hr_baseline, fi_1hr_baseline, predictions_1hr_baseline = run_monthly_rolling_forecast(baseline_data_1hr)

In [ ]:
results_1hr_generator_data, fi_1hr_generator_data, predictions_1hr_generator_data = run_monthly_rolling_forecast(generator_data_1hr) # This is for all generators (not specific planned/unplanned or top 50)

## 2hr Ahead Forecast

In [ ]:
results_2hr_baseline, fi_2hr_baseline, predictions_2hr_baseline = run_monthly_rolling_forecast(baseline_data_2hr)

In [ ]:
results_2hr_generator_data, fi_2hr_generator_data, predictions_2hr_generator_data = run_monthly_rolling_forecast(generator_data_2hr)

## 4hr Ahead Forecast

In [ ]:
results_4hr_baseline, fi_4hr_baseline, predictions_4hr_baseline = run_monthly_rolling_forecast(baseline_data_4hr)

In [ ]:
results_4hr_generator_data, fi_4hr_generator_data, predictions_4hr_generator_data = run_monthly_rolling_forecast(generator_data_4hr)

## 8hr Ahead Forecast

In [ ]:
results_8hr_baseline, fi_8hr_baseline, predictions_8hr_baseline = run_monthly_rolling_forecast(baseline_data_8hr)

In [ ]:
results_8hr_generator_data, fi_8hr_generator_data, predictions_8hr_generator_data = run_monthly_rolling_forecast(generator_data_8hr)

## 12hr Ahead Forecast

In [ ]:
results_12hr_baseline, fi_12hr_baseline, predictions_12hr_baseline = run_monthly_rolling_forecast(baseline_data_12hr)

In [ ]:
results_12hr_generator_data, fi_12hr_generator_data, predictions_12hr_generator_data = run_monthly_rolling_forecast(generator_data_12hr)